# Korea Imports - Analysis

## Dependencies

In [ ]:
import os
import sys
import duckdb

sys.path.append("../ingest")
import config # type: ignore

## Check for Files in Clean Data Directory

In [ ]:
clean_data = "." + config.Local_Dir.Korea_Imports["clean"]
print(os.listdir(clean_data))

## Duckdb Database

In [ ]:
db_dir = config.Database.dir
db_filename = config.Database.filename
db_path = os.path.join(db_dir, db_filename)

dataset = clean_data + config.datasets["korea-imports"]["filename-clean"]

if not os.path.exists(db_path):
    os.makedirs(db_dir, exist_ok=True)

ddb = duckdb.connect(database=db_path)

In [63]:
ddb.execute(
    """CREATE TABLE IF NOT EXISTS korea_imports
    AS SELECT * FROM '../data/korea-imports/clean/clean_koreaimports.parquet'"""
)

In [68]:
rel = ddb.sql(
    """SELECT cmdCode, SUM(primaryValue) total FROM korea_imports
    WHERE partnerDesc = 'Colombia'
    GROUP BY cmdCode
    ORDER BY total desc;"""
)
type(rel)

duckdb.duckdb.DuckDBPyRelation

## Dataset Summary

TODO:
- Update rel objects to views
- Relations are queries uppon a file or database!

## Top Korea Imports from World by Value (USD)

- MINERAL FUELS, MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES = **27**.
- ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS THEREOF; SOUND RECORDERS AND REPRODUCERS; TELEVISION IMAGE AND SOUND RECORDERS AND REPRODUCERS, PARTS AND ACCESSORIES OF SUCH ARTICLES = **85**.
- NUCLEAR REACTORS, BOILERS, MACHINERY AND MECHANICAL APPLIANCES; PARTS THEREOF = **84**.
- VEHICLES; OTHER THAN RAILWAY OR TRAMWAY ROLLING STOCK, AND PARTS AND ACCESSORIES THEREOF = **87**.
- OPTICAL, PHOTOGRAPHIC, CINEMATOGRAPHIC, MEASURING, CHECKING, MEDICAL OR SURGICAL INSTRUMENTS AND APPARATUS; PARTS AND ACCESSORIES = **90**.
- INORGANIC CHEMICALS; ORGANIC AND INORGANIC COMPOUNDS OF PRECIOUS METALS; OF RARE EARTH METALS, OF RADIO-ACTIVE ELEMENTS AND OF ISOTOPES = **28**.
- ORES, SLAG AND ASH = **26**.
- IRON AND STEEL = **72**.
- ORGANIC CHEMICALS = **29**.
- PLASTICS AND ARTICLES THEREOF = **39**.

In [ ]:
rel.query(
    virtual_table_name="top_world_imports",
    sql_query="""SELECT cmdCode, SUM(primaryValue) totalValue
    FROM rel
    WHERE partnerDesc = 'World'
    GROUP BY cmdCode
    ORDER BY totalValue desc
    LIMIT 10;"""
)

## Top Korea Import from China and USA by Value (USD)

- ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS THEREOF; SOUND RECORDERS AND REPRODUCERS; TELEVISION IMAGE AND SOUND RECORDERS AND REPRODUCERS, PARTS AND ACCESSORIES OF SUCH ARTICLES = **85**.
- NUCLEAR REACTORS, BOILERS, MACHINERY AND MECHANICAL APPLIANCES; PARTS THEREOF = **84**.
- MINERAL FUELS, MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES = **27**.
- INORGANIC CHEMICALS; ORGANIC AND INORGANIC COMPOUNDS OF PRECIOUS METALS; OF RARE EARTH METALS, OF RADIO-ACTIVE ELEMENTS AND OF ISOTOPES = **28**.
- OPTICAL, PHOTOGRAPHIC, CINEMATOGRAPHIC, MEASURING, CHECKING, MEDICAL OR SURGICAL INSTRUMENTS AND APPARATUS; PARTS AND ACCESSORIES = **90**.
- VEHICLES; OTHER THAN RAILWAY OR TRAMWAY ROLLING STOCK, AND PARTS AND ACCESSORIES THEREOF = **87**.
- IRON AND STEEL = **72**.
- ORGANIC CHEMICALS = **29**.
- PLASTICS AND ARTICLES THEREOF = **39**.
- IRON OR STEEL ARTICLES = **73**.

In [ ]:
rel.query(
    virtual_table_name="top_partners_imports",
    sql_query="""SELECT cmdCode, SUM(primaryValue) totalValue
    FROM rel
    WHERE partnerDesc IN ('China', 'USA')
    GROUP BY cmdCode
    ORDER BY totalValue desc
    LIMIT 10;"""
)

## Top Korea Imports form Iberoamerican Partners by Value (USD)

- ORES, SLAG AND ASH = **26**.
- MINERAL FUELS, MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES = **27**.
- INORGANIC CHEMICALS; ORGANIC AND INORGANIC COMPOUNDS OF PRECIOUS METALS; OF RARE EARTH METALS, OF RADIO-ACTIVE ELEMENTS AND OF ISOTOPES = **28**.
- CEREALS = **10**.
- COPPER AND ARTICLES THEREOF = **74**.
- NUCLEAR REACTORS, BOILERS, MACHINERY AND MECHANICAL APPLIANCES; PARTS THEREOF = **84**.
- VEHICLES; OTHER THAN RAILWAY OR TRAMWAY ROLLING STOCK, AND PARTS AND ACCESSORIES THEREOF = **87**.
- FOOD INDUSTRIES, RESIDUES AND WASTES THEREOF; PREPARED ANIMAL FODDER = **23**.
- ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS THEREOF; SOUND RECORDERS AND REPRODUCERS; TELEVISION IMAGE AND SOUND RECORDERS AND REPRODUCERS, PARTS AND ACCESSORIES OF SUCH ARTICLES = **85**.
- MEAT AND EDIBLE MEAT OFFAL = **02**.



In [ ]:
rel.query(
    virtual_table_name="top_iberoamerican_imports",
    sql_query="""SELECT cmdCode, SUM(primaryValue) totalValue
    FROM rel
    WHERE partnerDesc NOT IN ('World', 'China', 'USA', 'Colombia')
    GROUP BY cmdCode
    ORDER BY totalValue desc
    LIMIT 10;"""
)

## Top Korea Imports from Colombia by Value (USD)

- MINERAL FUELS, MINERAL OILS AND PRODUCTS OF THEIR DISTILLATION; BITUMINOUS SUBSTANCES; MINERAL WAXES = **27**.
- COFFEE, TEA, MATE AND SPICES = **09**.
- IRON AND STEEL = **72**.
- COPPER AND ARTICLES THEREOF = **74**.
- TREES AND OTHER PLANTS, LIVE; BULBS, ROOTS AND THE LIKE; CUT FLOWERS AND ORNAMENTAL FOLIAGE = **06**.
- MISCELLANEOUS EDIBLE PREPARATIONS = **21**.
- CHEMICAL PRODUCTS N.E.C. = **38**.
- FRUIT AND NUTS, EDIBLE; PEEL OF CITRUS FRUIT OR MELONS = **08**.
- ORES, SLAG AND ASH = **26**.
- ALUMINIUM AND ARTICLES THEREOF = **76**.

In [ ]:
rel.query(
    virtual_table_name="top_colombia_imports",
    sql_query="""SELECT cmdCode, SUM(primaryValue) totalValue
    FROM rel
    WHERE partnerDesc = 'Colombia'
    GROUP BY cmdCode
    ORDER BY totalValue desc
    LIMIT 10;"""
)

In [ ]:
conn.sql("show all tables")

In [ ]:
ddb.execute("CREATE TABLE test AS SELECT * FROM rel;")